In [3]:
import pandas as pd
import numpy as np
import matplotlib as plt
import scipy 
import seaborn as sns
from typing import *
import pyodbc
import sqlalchemy

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

In [7]:
# Leyendo Querys

def get_mssql_conn(server_name: str = 'PAUTGSQLP43'):
    return pyodbc.connect('DRIVER={};SERVER={};TRUSTED_CONNECTION=YES;'.format('SQL SERVER', server_name))

Q_PRACTICANTES = '''
    SELECT 
	A.periodo, 
	A.Cod_SIGA, 
	A.STATUS, 
	A.Matricula, 
	A.Fec_Ingreso, 
	A.tiempo_bcp, 
	A.Cod_GG, 
	A.Reporte_GG, 
	A.Cod_Division, 
	A.Nombre_Division, 
	A.Cod_Area, 
	A.Nombre_Area, 
	A.Cod_Servicio, 
	A.Nombre_Servicio, 
	A.Cod_UO, 
	A.Nombre_UO, 
	A.Sexo, 
	A.Edad, 
	A.Rango_Edad,
	CASE
		WHEN B.NUM_DOCUMENTO IS NULL THEN 0
		ELSE 1
	END AS 'Target'
    FROM BCP_GDH_PA_DW.GENERAL.BASE_COMPLETA A
        LEFT JOIN BCP_GDH_PA_DW.GENERAL.D_COLABORADOR B 
            ON A.Num_Doc = B.NUM_DOCUMENTO AND B.TIPO_PREPER = 'Orgánico' AND DATEDIFF(MONTH, B.FEC_INGRESO_BANCO, A.Fec_Cese) BETWEEN 0 AND 12
    WHERE 
        A.TIPO_PREPER = 'Practicante' AND 
        A.Sociedad = 'BCP Perú' AND 
        A.nxt_Tipo_PrePer = 'ultimo registro'
    ORDER BY A.periodo DESC
'''

def read_data() -> pd.DataFrame:
    conn = get_mssql_conn()
    return pd.read_sql_query(Q_PRACTICANTES, conn)

df_data = read_data()

df_data

,periodo,Cod_SIGA,STATUS,Matricula,Fec_Ingreso,tiempo_bcp,Cod_GG,Reporte_GG,Cod_Division,Nombre_Division,Cod_Area,Nombre_Area,Cod_Servicio,Nombre_Servicio,Cod_UO,Nombre_UO,Sexo,Edad,Rango_Edad,Target
0,202408,00639266,ACTIVO,0T39266,2023-09-15,0.963723,00019957,GCIA GENERAL ADJ DE TRANSF E INNOVACION,00022577,DIVISION YAPE,00030531,TRIBU YAPE LENDING,00030531,TRIBU YAPE LENDING,00025787,SQ LENDING - PERSONAS,Masculino,23,1. Hasta 25,0
1,202408,00650627,ACTIVO,0T50627,2024-07-15,0.131416,00019957,GCIA GENERAL ADJ DE TRANSF E INNOVACION,00022577,DIVISION YAPE,00022580,TRIBU YAPE EXPERIENCIA,00022580,TRIBU YAPE EXPERIENCIA,00023628,SQ RECLAMOS Y CX,Masculino,22,1. Hasta 25,0
2,202408,00631887,ACTIVO,0T31887,2023-02-01,1.582477,00000534,"GERENCIA CENTRAL DE TI, OPERA Y DATA",00000534,"GERENCIA CENTRAL DE TI, OPERA Y DATA",00025237,"GCIA AREA ESTRAT. FINANZAS TI, DATA Y OP",00018383,GCIA DE GESTIÓN FINAN. Y VENDOR MANAGEME,00021052,VENDOR PERFORMANCE & ACCOUTING,Masculino,21,1. Hasta 25,0
3,202408,00640613,ACTIVO,0T40613,2023-11-01,0.835044,00002620,GERENCIA CENTRAL DE RIESGOS,00000267,GERENCIA DE DIVISION DE CREDITOS,00000267,GERENCIA DE DIVISION DE CREDITOS,00003052,GCIA DE PLANEAMIENTO DE CREDITOS,00026659,GESTION REGULATORIA E INDICADORES,Masculino,25,1. Hasta 25,0
4,202408,00651518,ACTIVO,0T51518,2024-08-15,0.046543,00000298,GER.CTRAL PLANEAMIENTO Y FINANZAS,00000429,GCIA DE DIVISION DE CONTABILIDAD GENERAL,00000429,GCIA DE DIVISION DE CONTABILIDAD GENERAL,00000203,GERENCIA CONTABILIDAD CONTROL INTERNO,00000203,GERENCIA CONTABILIDAD CONTROL INTERNO,Masculino,20,1. Hasta 25,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5027,201101,00508263,CESADO,0S08263,2010-06-01,0.585900,00002620,GERENCIA CENTRAL DE RIESGOS,00000007,GERENCIA DE DIVISIÓN SOLUCIONES DE PAGO,00000007,GERENCIA DE DIVISIÓN SOLUCIONES DE PAGO,00002780,GCIA. ÁREA ESTRATEGIA Y DESEMPEÑO DE SDP,00002728,SUBGERENCIA DE TECNOLOGÍA SDP,Masculino,23,1. Hasta 25,0
5028,201101,00507813,CESADO,0S07813,2010-04-05,0.741957,00000534,"GERENCIA CENTRAL DE TI, OPERA Y DATA",00000534,"GERENCIA CENTRAL DE TI, OPERA Y DATA",00000535,GERENCIA DE ÁREA OPERACIONES Y PROCESOS,00000164,TRIBU POST VENTA Y GESTION TRANSACCIONAL,00002215,SQ GUNGNIR,Masculino,21,1. Hasta 25,0
5029,201101,00506411,CESADO,0S06411,2009-09-01,1.333333,00011951,GERENCIA DE DIVISION ASUNTOS CORPORATIVO,00011951,GERENCIA DE DIVISION ASUNTOS CORPORATIVO,00011951,GERENCIA DE DIVISION ASUNTOS CORPORATIVO,00003053,GERENCIA RESP SOCIAL Y RELACIONES PUBLIC,00003053,GERENCIA RESP SOCIAL Y RELACIONES PUBLIC,Femenino,24,1. Hasta 25,0
5030,201101,00507077,CESADO,0S07077,2009-12-01,1.084188,00002620,GERENCIA CENTRAL DE RIESGOS,00000267,GERENCIA DE DIVISION DE CREDITOS,00000268,GCIA DE AREA CUENT ESPEC Y SEG DE CREDIT,00000273,GERENCIA DE NEGOCIOS LIMA II,00000273,GERENCIA DE NEGOCIOS LIMA II,Femenino,23,1. Hasta 25,0


In [8]:
df_data.head(10)
df_data.tail(10)

,periodo,Cod_SIGA,STATUS,Matricula,Fec_Ingreso,tiempo_bcp,Cod_GG,Reporte_GG,Cod_Division,Nombre_Division,Cod_Area,Nombre_Area,Cod_Servicio,Nombre_Servicio,Cod_UO,Nombre_UO,Sexo,Edad,Rango_Edad,Target
5022,201101,00506874,CESADO,0S06874,2009-11-01,1.166324,00000534,"GERENCIA CENTRAL DE TI, OPERA Y DATA",00000534,"GERENCIA CENTRAL DE TI, OPERA Y DATA",00000535,GERENCIA DE ÁREA OPERACIONES Y PROCESOS,00028467,COE BUSINESS CONSULTING & PROCESS AUTOMA,00002105,DPTO.PLANIFICACION DE PROCESO MINORISTA,Femenino,23,1. Hasta 25,0
5023,201101,00507233,CESADO,0S07233,2010-01-04,1.056810,00000378,GCIA CENTRAL DE EMPRESAS Y CORPORACIONES,00000125,GERENCIA DE DIVISION BANCA EMPRESA,00000289,AREA BANCA EMPRESAS LIMA 2 Y BANCA INST.,00000286,GERENCIA DE NEGOCIOS MULTINACIONAL,00000286,GERENCIA DE NEGOCIOS MULTINACIONAL,Masculino,22,1. Hasta 25,1
5024,201101,00505720,CESADO,0S05720,2009-05-01,1.735797,00000378,GCIA CENTRAL DE EMPRESAS Y CORPORACIONES,00000125,GERENCIA DE DIVISION BANCA EMPRESA,00003035,AREA BANCA EMPRESAS LIMA(3),00003037,SERV. GRUPO DE NEG. CRONOS II,00003037,SERV. GRUPO DE NEG. CRONOS II,Masculino,23,1. Hasta 25,1
5025,201101,00507078,CESADO,0S07078,2009-12-01,1.086926,00000211,GCIA DE DIV ESTRATEGIA DE PERSONAS Y ADM,00000211,GCIA DE DIV ESTRATEGIA DE PERSONAS Y ADM,00000221,GCIA AREA EXPERIENCIA DEL COLABORADOR,00000229,GERENCIA ENTRENAMIENTO Y CAPACITACIÓN,00000229,GERENCIA ENTRENAMIENTO Y CAPACITACIÓN,Masculino,21,1. Hasta 25,0
5026,201101,00508024,CESADO,0S08024,2010-05-03,0.665297,00000378,GCIA CENTRAL DE EMPRESAS Y CORPORACIONES,00000125,GERENCIA DE DIVISION BANCA EMPRESA,00003035,AREA BANCA EMPRESAS LIMA(3),00003036,SERV. GRUPO DE NEG. CRONOS I,00003036,SERV. GRUPO DE NEG. CRONOS I,Masculino,22,1. Hasta 25,0
5027,201101,00508263,CESADO,0S08263,2010-06-01,0.585900,00002620,GERENCIA CENTRAL DE RIESGOS,00000007,GERENCIA DE DIVISIÓN SOLUCIONES DE PAGO,00000007,GERENCIA DE DIVISIÓN SOLUCIONES DE PAGO,00002780,GCIA. ÁREA ESTRATEGIA Y DESEMPEÑO DE SDP,00002728,SUBGERENCIA DE TECNOLOGÍA SDP,Masculino,23,1. Hasta 25,0
5028,201101,00507813,CESADO,0S07813,2010-04-05,0.741957,00000534,"GERENCIA CENTRAL DE TI, OPERA Y DATA",00000534,"GERENCIA CENTRAL DE TI, OPERA Y DATA",00000535,GERENCIA DE ÁREA OPERACIONES Y PROCESOS,00000164,TRIBU POST VENTA Y GESTION TRANSACCIONAL,00002215,SQ GUNGNIR,Masculino,21,1. Hasta 25,0
5029,201101,00506411,CESADO,0S06411,2009-09-01,1.333333,00011951,GERENCIA DE DIVISION ASUNTOS CORPORATIVO,00011951,GERENCIA DE DIVISION ASUNTOS CORPORATIVO,00011951,GERENCIA DE DIVISION ASUNTOS CORPORATIVO,00003053,GERENCIA RESP SOCIAL Y RELACIONES PUBLIC,00003053,GERENCIA RESP SOCIAL Y RELACIONES PUBLIC,Femenino,24,1. Hasta 25,0
5030,201101,00507077,CESADO,0S07077,2009-12-01,1.084188,00002620,GERENCIA CENTRAL DE RIESGOS,00000267,GERENCIA DE DIVISION DE CREDITOS,00000268,GCIA DE AREA CUENT ESPEC Y SEG DE CREDIT,00000273,GERENCIA DE NEGOCIOS LIMA II,00000273,GERENCIA DE NEGOCIOS LIMA II,Femenino,23,1. Hasta 25,0
5031,201101,00507466,CESADO,0S07466,2010-02-01,0.914442,00000534,"GERENCIA CENTRAL DE TI, OPERA Y DATA",00000534,"GERENCIA CENTRAL DE TI, OPERA Y DATA",00000535,GERENCIA DE ÁREA OPERACIONES Y PROCESOS,00000542,GCIA DE ADMINISTRACION DE PROCEDIMIENTOS,00000542,GCIA DE ADMINISTRACION DE PROCEDIMIENTOS,Femenino,23,1. Hasta 25,0


In [11]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5032 entries, 0 to 5031
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   periodo          5032 non-null   int64  
 1   Cod_SIGA         5032 non-null   object 
 2   STATUS           5032 non-null   object 
 3   Matricula        5032 non-null   object 
 4   Fec_Ingreso      5032 non-null   object 
 5   tiempo_bcp       5032 non-null   float64
 6   Cod_GG           5032 non-null   object 
 7   Reporte_GG       5032 non-null   object 
 8   Cod_Division     5032 non-null   object 
 9   Nombre_Division  5032 non-null   object 
 10  Cod_Area         5032 non-null   object 
 11  Nombre_Area      5032 non-null   object 
 12  Cod_Servicio     5032 non-null   object 
 13  Nombre_Servicio  5032 non-null   object 
 14  Cod_UO           5032 non-null   object 
 15  Nombre_UO        5032 non-null   object 
 16  Sexo             5032 non-null   object 
 17  Edad          

In [12]:
df_data.describe()

,periodo,tiempo_bcp,Edad,Target
count,5032.000000,5032.000000,5032.000000,5032.000000
mean,201806.038951,0.909039,22.686010,0.105326
std,427.002571,0.576430,1.759428,0.307003
min,201101.000000,0.002737,18.000000,0.000000
25%,201407.000000,0.495550,21.750000,0.000000
50%,201807.000000,0.799452,22.000000,0.000000
75%,202206.250000,1.185489,24.000000,0.000000
max,202408.000000,6.921286,36.000000,1.000000


In [13]:
df_data.shape

(5032, 20)

In [14]:
df_data.dtypes

periodo              int64
Cod_SIGA            object
STATUS              object
Matricula           object
Fec_Ingreso         object
tiempo_bcp         float64
Cod_GG              object
Reporte_GG          object
Cod_Division        object
Nombre_Division     object
Cod_Area            object
Nombre_Area         object
Cod_Servicio        object
Nombre_Servicio     object
Cod_UO              object
Nombre_UO           object
Sexo                object
Edad                 int64
Rango_Edad          object
Target               int64
dtype: object

In [15]:
df_data.isnull().sum()

periodo            0
Cod_SIGA           0
STATUS             0
Matricula          0
Fec_Ingreso        0
tiempo_bcp         0
Cod_GG             0
Reporte_GG         0
Cod_Division       0
Nombre_Division    0
Cod_Area           0
Nombre_Area        0
Cod_Servicio       0
Nombre_Servicio    0
Cod_UO             0
Nombre_UO          0
Sexo               0
Edad               0
Rango_Edad         0
Target             0
dtype: int64

In [16]:
df_data.nunique()

periodo             164
Cod_SIGA           5031
STATUS                2
Matricula          5031
Fec_Ingreso         786
tiempo_bcp          754
Cod_GG               16
Reporte_GG           16
Cod_Division         37
Nombre_Division      37
Cod_Area            163
Nombre_Area         163
Cod_Servicio        400
Nombre_Servicio     400
Cod_UO              889
Nombre_UO           884
Sexo                  2
Edad                 17
Rango_Edad            3
Target                2
dtype: int64

In [21]:
df_data.duplicated().sum()

0

In [23]:
DIC_COL_PRACTIS = {
    'Cod_SIGA': 'COD_SIGA',
    'Matricula': 'MATRICULA',
    'Fec_Ingreso': 'FECHA_INGRESO',
    'Cod_GG' : 'CÓDIGO_GERENCIA_GENERAL',
    'Cod_Division': 'COD_DIVISION',
    'Cod_Area' : 'COD_AREA',
    'Sexo' : 'GENERO',
    'Edad' : 'EDAD',
    'Rango_Edad' : 'RANGO_EDAD',
    'Target' : 'TARGET'
}

def columnas(df: pd.DataFrame, dic_cols: Dict[str, str]):
    df = df.rename(columns=dic_cols)
    arr_cols = [v for k,v in dic_cols.items()]
    df = df[arr_cols]
    return df

df_data = columnas(df_data, DIC_COL_PRACTIS)


In [18]:
df_data.drop_duplicates(inplace=True)

In [24]:
df_data.drop(columns=['COD_SIGA'], inplace=True)

In [26]:
df_data.dropna(inplace=True)

In [28]:
columns_castear = {
    'FECHA_INGRESO': 'datetime64',
}
df_data = df_data.astype(columns_castear)

In [29]:
df_data.nunique().sort_values(ascending=True)

GENERO                        2
TARGET                        2
RANGO_EDAD                    3
CÓDIGO_GERENCIA_GENERAL      16
EDAD                         17
COD_DIVISION                 37
COD_AREA                    163
FECHA_INGRESO               786
MATRICULA                  5031
dtype: int64

In [31]:
df_data.duplicated().sum()

0

In [32]:
df_data.isnull()

,MATRICULA,FECHA_INGRESO,CÓDIGO_GERENCIA_GENERAL,COD_DIVISION,COD_AREA,GENERO,EDAD,RANGO_EDAD,TARGET
0,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
5027,False,False,False,False,False,False,False,False,False
5028,False,False,False,False,False,False,False,False,False
5029,False,False,False,False,False,False,False,False,False
5030,False,False,False,False,False,False,False,False,False


In [33]:
df_data.isnull().sum()

MATRICULA                  0
FECHA_INGRESO              0
CÓDIGO_GERENCIA_GENERAL    0
COD_DIVISION               0
COD_AREA                   0
GENERO                     0
EDAD                       0
RANGO_EDAD                 0
TARGET                     0
dtype: int64